In [8]:
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np
import itertools
import random
import copy
import pandas as pd
import networkx as nx
from matplotlib import animation
import seaborn as sns
plt.style.use("seaborn-dark-palette")

In [9]:
def create_agent_data_fixed_groups(num_eco,num_trad,num_normal) -> int:
    #load schedhule and load data
    data = pd.read_excel(r'notebooks/data/workspace_files/basic_load_profile.xlsx')
    data = data.set_index('time')
    #store data in schedule dictionary key=hour, pair= load [kW]
    schedule = {}
    susceptibility = 0.1
    group_weight = 0.1

    for index, row in data.iterrows():
        schedule[index] = row['load_1']
    print(schedule)

    # create groups
    groups = ['eco', 'normal', 'trad']
    # create dataframe to store agent info
    data_frame = pd.DataFrame(columns= ['id'])

    # for each agent add following attributes
    for agent in range(num_eco):
        group = groups[0]
        group_weight = group_weight
        flexibility_coeficient = 0.8
        conv_weight = 0.5
        price_weight = 0.5

        data = {
            'schedule': schedule,
            'group_weight': group_weight,
            'flexibility_coeficient': flexibility_coeficient,
            'susceptibility': susceptibility,
            'group': group,
            #'group_weight': group_weight,
            'shifted_hours': 0,
            'conv_weight': conv_weight,
            'price_weight': price_weight
        }
        data_frame = data_frame.append(data, ignore_index=True)

    for agent in range(num_normal):
        group = groups[1]
        group_weight = group_weight
        flexibility_coeficient = 0.5
        conv_weight = 1
        price_weight = 0.5

        data = {
            'schedule': schedule,
            'group_weight': group_weight,
            'flexibility_coeficient': flexibility_coeficient,
            'susceptibility': susceptibility,
            'group': group,
            #'group_weight': group_weight,
            'shifted_hours': 0,
            'conv_weight': conv_weight,
            'price_weight': price_weight
            }
        data_frame = data_frame.append(data, ignore_index=True)

    for agent in range(num_trad):
        group = groups[2]
        group_weight = group_weight
        flexibility_coeficient = 0.3
        conv_weight = 1
        price_weight = 1

        data = {'schedule': schedule,
                'group_weight': group_weight,
                'flexibility_coeficient': flexibility_coeficient,
                'susceptibility': susceptibility,
                'group': group,
                #'group_weight': group_weight,
                'shifted_hours': 0,
                'conv_weight': conv_weight,
                'price_weight': price_weight
                }


        data_frame = data_frame.append(data, ignore_index=True)

    data_frame.insert(0,'New_id', range(0,len(data_frame)))
    #print(data_frame.index.tolist())
    return data_frame.set_index('New_id')# Return dataframve with index ID

In [7]:
# prepare data with function create_agent_data and save it in variable
#new_agent_data = create_agent_data(12)
new_agent_data = create_agent_data_fixed_groups(4,4,4)
new_agent_data.to_csv('random_agentdata.csv')
new_agent_data

{1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.2, 6: 0.1, 7: 0.1, 8: 0.1, 9: 0.1, 10: 0.1, 11: 0.1, 12: 0.1, 13: 0.1, 14: 0.1, 15: 1.5, 16: 1.5, 17: 0.1, 18: 2.0, 19: 3.0, 20: 2.0, 21: 1.0, 22: 0.2, 23: 0.2, 24: 0.1}


,id,conv_weight,flexibility_coeficient,group,group_weight,price_weight,schedule,shifted_hours,susceptibility
New_id,,,,,,,,,
0,NaN,0.5,0.8,eco,0.1,0.5,"{1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.2, 6: 0....",0.0,0.1
1,NaN,0.5,0.8,eco,0.1,0.5,"{1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.2, 6: 0....",0.0,0.1
2,NaN,0.5,0.8,eco,0.1,0.5,"{1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.2, 6: 0....",0.0,0.1
3,NaN,0.5,0.8,eco,0.1,0.5,"{1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.2, 6: 0....",0.0,0.1
4,NaN,1.0,0.5,normal,0.1,0.5,"{1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.2, 6: 0....",0.0,0.1
5,NaN,1.0,0.5,normal,0.1,0.5,"{1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.2, 6: 0....",0.0,0.1
6,NaN,1.0,0.5,normal,0.1,0.5,"{1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.2, 6: 0....",0.0,0.1
7,NaN,1.0,0.5,normal,0.1,0.5,"{1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.2, 6: 0....",0.0,0.1
8,NaN,1.0,0.3,trad,0.1,1.0,"{1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.2, 6: 0....",0.0,0.1
